In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import optuna

# 
# ==========================================
# 0. CONFIGURATION & DEVICE
# ==========================================
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f">>> Running on: {DEVICE}")

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

FEATURE_COUNTS = [500, 1500, 2000, 2500, 3000]
SUBTYPES_OF_INTEREST = [
    'Leiomyosarcoma, NOS',
    'Dedifferentiated liposarcoma',
    'Undifferentiated sarcoma',
    'Fibromyxosarcoma'
]
N_TRIALS = 15

>>> Running on: cuda


In [7]:
def load_raw_aligned_data():
    print(f"\n>>> LOADING RAW ALIGNED DATA")
    
    # 1. Load Phenotype/Labels
    pheno_path = "../Data/phenotype_clean.csv"
    if not os.path.exists(pheno_path):
        raise FileNotFoundError(f"{pheno_path} not found.")
    
    pheno = pd.read_csv(pheno_path, index_col=0)
    
    col_name = 'primary_diagnosis.diagnoses'
    if col_name not in pheno.columns:
        print(f"Warning: '{col_name}' not found.")
        return None
        
    mask = pheno[col_name].isin(SUBTYPES_OF_INTEREST)
    pheno = pheno[mask]
    print(f"  Phenotype Samples (filtered): {pheno.shape[0]}")

    # 2. Load Omics
    def load_omic(path, name):
        if not os.path.exists(path):
            print(f"Warning: {path} not found.")
            return None
        df = pd.read_csv(path, index_col=0)
        df = df.T # samples x features
        return df

    rna = load_omic("../Data/expression_log.csv", "RNA (Expression)")
    meth = load_omic("../Data/methylation_mvalues.csv", "Methylation")
    cnv = load_omic("../Data/cnv_log.csv", "CNV")
    
    if rna is None or meth is None or cnv is None:
        raise ValueError("One or more omics files missing.")

    # 3. Intersection
    common_samples = pheno.index.intersection(rna.index).intersection(meth.index).intersection(cnv.index)
    print(f"  Common Samples: {len(common_samples)}")
    
    if len(common_samples) == 0:
        raise ValueError("No common samples found!")

    pheno = pheno.loc[common_samples]
    rna = rna.loc[common_samples]
    meth = meth.loc[common_samples]
    cnv = cnv.loc[common_samples]
    
    # 4. Prepare Labels
    le = LabelEncoder()
    Y = le.fit_transform(pheno[col_name])
    print(f"  Classes: {le.classes_}")

    return rna, meth, cnv, Y, le.classes_

# 

In [8]:
class LowRankHyperNetwork(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, in_features, out_features, rank=16):
        super(LowRankHyperNetwork, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.rank = rank
        
        self.backbone = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )
        
        self.U_generator = nn.Linear(hidden_dim, in_features * rank)
        self.V_generator = nn.Linear(hidden_dim, rank * out_features)
        self.bias_generator = nn.Linear(hidden_dim, out_features)
        self.scale = nn.Parameter(torch.ones(1) * 0.1)
    
    def forward(self, embedding):
        features = self.backbone(embedding)
        U = self.U_generator(features).view(self.in_features, self.rank)
        V = self.V_generator(features).view(self.rank, self.out_features)
        weight = (U @ V).T * self.scale
        bias = self.bias_generator(features)
        return weight, bias

class HyperEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, embedding_dim, 
                 hyper_hidden_dim=64, rank=16, dropout=0.3):
        super(HyperEncoder, self).__init__()
        self.hyper_layer = LowRankHyperNetwork(
            embedding_dim=embedding_dim,
            hidden_dim=hyper_hidden_dim,
            in_features=input_dim,
            out_features=hidden_dims[0],
            rank=rank
        )
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        prev_dim = hidden_dims[0]
        for hidden_dim in hidden_dims[1:]:
            self.layers.append(nn.Linear(prev_dim, hidden_dim))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))
            prev_dim = hidden_dim
        self.final_layer = nn.Linear(prev_dim, latent_dim)
        self.first_bn = nn.BatchNorm1d(hidden_dims[0])
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, embedding):
        weight, bias = self.hyper_layer(embedding)
        x = F.linear(x, weight, bias)
        x = self.first_bn(x)
        x = F.relu(x)
        x = self.dropout(x)
        for layer, bn in zip(self.layers, self.batch_norms):
            x = layer(x) # bn(layer(x)) usually better, adjusted order here
            x = bn(x)
            x = F.relu(x)
            x = self.dropout(x)
        z = self.final_layer(x)
        return z

class HyperDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dims, output_dim, dropout=0.3):
        super(HyperDecoder, self).__init__()
        layers = []
        prev_dim = latent_dim
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.decoder = nn.Sequential(*layers)
    
    def forward(self, z):
        return self.decoder(z)

class HyperDNN(nn.Module):
    def __init__(self, input_dims, latent_dim=32, hyper_hidden_dim=64, 
                 embedding_dim=16, encoder_hidden_dims=[128, 64], 
                 rank=16, dropout=0.4):
        super(HyperDNN, self).__init__()
        self.input_dims = input_dims
        self.latent_dim = latent_dim
        self.omics_names = list(input_dims.keys())
        self.num_omics = len(self.omics_names)
        self.embedding_dim = embedding_dim
        self.omics_embeddings = nn.Embedding(self.num_omics, embedding_dim)
        self.encoders = nn.ModuleDict()
        self.decoders = nn.ModuleDict()
        for name, input_dim in input_dims.items():
            self.encoders[name] = HyperEncoder(
                input_dim=input_dim,
                hidden_dims=encoder_hidden_dims,
                latent_dim=latent_dim,
                embedding_dim=embedding_dim,
                hyper_hidden_dim=hyper_hidden_dim,
                rank=rank,
                dropout=dropout
            )
            self.decoders[name] = HyperDecoder(
                latent_dim=latent_dim,
                hidden_dims=list(reversed(encoder_hidden_dims)),
                output_dim=input_dim,
                dropout=dropout
            )
        for i, name in enumerate(self.omics_names):
            self.register_buffer(f'omics_idx_{name}', torch.tensor(i))
    
    def get_omics_embedding(self, omics_name, device):
        idx = getattr(self, f'omics_idx_{omics_name}')
        return self.omics_embeddings(idx)
    
    def encode(self, x, omics_name):
        embedding = self.get_omics_embedding(omics_name, x.device)
        z = self.encoders[omics_name](x, embedding)
        return z
    
    def decode(self, z, omics_name):
        return self.decoders[omics_name](z)
    
    def forward(self, inputs):
        latents = {}
        reconstructions = {}
        for name, x in inputs.items():
            z = self.encode(x, name)
            recon = self.decode(z, name)
            latents[name] = z
            reconstructions[name] = recon
        return latents, reconstructions

class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# 

In [9]:
def run_cv_evaluation(params, n_features, rna_df, meth_df, cnv_df, Y, class_names, is_optuna=True):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    fold_metrics = {'accuracy': [], 'f1_macro': [], 'f1_micro': [], 'precision': [], 'recall': []}
    
    # Hyperparams extraction
    latent_dim = params['latent_dim']
    hyper_hidden = params['hyper_hidden_dim']
    embed_dim = params['embedding_dim']
    rank = params['rank']
    enc_layers = params['n_encoder_layers']
    # Dynamic hidden dims
    enc_hidden = []
    current_dim = 256
    for _ in range(enc_layers):
        enc_hidden.append(current_dim)
        current_dim = current_dim // 2
    
    lr_ae = params['lr_ae']
    lr_clf = params['lr_clf']
    dropout = params['dropout']
    epochs_ae = 50 
    epochs_clf = 200

    for fold, (train_idx, val_idx) in enumerate(kf.split(rna_df, Y)):
        # Data Splitting
        tr_r, val_r = rna_df.iloc[train_idx], rna_df.iloc[val_idx]
        tr_m, val_m = meth_df.iloc[train_idx], meth_df.iloc[val_idx]
        tr_c, val_c = cnv_df.iloc[train_idx], cnv_df.iloc[val_idx]
        
        # Imputation
        imp = SimpleImputer(strategy='median')
        tr_r_imp = imp.fit_transform(tr_r); val_r_imp = imp.transform(val_r)
        tr_m_imp = imp.fit_transform(tr_m); val_m_imp = imp.transform(val_m)
        tr_c_imp = imp.fit_transform(tr_c); val_c_imp = imp.transform(val_c)
        
        # Variance Filter
        def get_top_k(d, k):
            v = np.var(d, axis=0)
            return np.argpartition(v, -k)[-k:] if d.shape[1] > k else np.arange(d.shape[1])
            
        r_idx = get_top_k(tr_r_imp, n_features)
        m_idx = get_top_k(tr_m_imp, n_features)
        c_idx = get_top_k(tr_c_imp, n_features)
        
        tr_r_sel = tr_r_imp[:, r_idx]; val_r_sel = val_r_imp[:, r_idx]
        tr_m_sel = tr_m_imp[:, m_idx]; val_m_sel = val_m_imp[:, m_idx]
        tr_c_sel = tr_c_imp[:, c_idx]; val_c_sel = val_c_imp[:, c_idx]
        
        # Scaling
        sc = StandardScaler()
        tr_r_fin = sc.fit_transform(tr_r_sel); val_r_fin = sc.transform(val_r_sel)
        tr_m_fin = sc.fit_transform(tr_m_sel); val_m_fin = sc.transform(val_m_sel)
        tr_c_fin = sc.fit_transform(tr_c_sel); val_c_fin = sc.transform(val_c_sel)
        
        # Tensors
        inputs_tr = {
            'RNA': torch.FloatTensor(tr_r_fin).to(DEVICE),
            'Meth': torch.FloatTensor(tr_m_fin).to(DEVICE),
            'CNV': torch.FloatTensor(tr_c_fin).to(DEVICE)
        }
        inputs_val = {
            'RNA': torch.FloatTensor(val_r_fin).to(DEVICE),
            'Meth': torch.FloatTensor(val_m_fin).to(DEVICE),
            'CNV': torch.FloatTensor(val_c_fin).to(DEVICE)
        }
        y_tr = torch.LongTensor(Y[train_idx]).to(DEVICE)
        y_val = torch.LongTensor(Y[val_idx]).to(DEVICE)
        
        input_dims = {'RNA': tr_r_fin.shape[1], 'Meth': tr_m_fin.shape[1], 'CNV': tr_c_fin.shape[1]}
        
        # --- A. Train HyperDNN (Autoencoder) ---
        model_ae = HyperDNN(
            input_dims=input_dims,
            latent_dim=latent_dim,
            hyper_hidden_dim=hyper_hidden,
            embedding_dim=embed_dim,
            encoder_hidden_dims=enc_hidden,
            rank=rank,
            dropout=dropout
        ).to(DEVICE)
        
        opt_ae = optim.AdamW(model_ae.parameters(), lr=lr_ae)
        crit_mse = nn.MSELoss()
        
        model_ae.train()
        for e in range(epochs_ae):
            opt_ae.zero_grad()
            _, recons = model_ae(inputs_tr)
            loss = 0
            for k in inputs_tr:
                loss += crit_mse(recons[k], inputs_tr[k])
            loss.backward()
            opt_ae.step()
            
        # --- B. Train Classifier (on Latents) ---
        model_ae.eval()
        with torch.no_grad():
            lat_tr, _ = model_ae(inputs_tr)
            lat_val, _ = model_ae(inputs_val)
            
            # Fuse = Concat
            z_tr = torch.cat([lat_tr[k] for k in ['RNA', 'Meth', 'CNV']], dim=1)
            z_val = torch.cat([lat_val[k] for k in ['RNA', 'Meth', 'CNV']], dim=1)
            
        clf = SimpleClassifier(z_tr.shape[1], 128, len(np.unique(Y)), dropout).to(DEVICE)
        opt_clf = optim.AdamW(clf.parameters(), lr=lr_clf, weight_decay=params.get('weight_decay', 1e-4))
        
        # Calculate weights based on training labels only (to avoid leakage)
        unique_classes = np.unique(Y[train_idx])
        weights = compute_class_weight(
            class_weight='balanced', 
            classes=unique_classes, 
            y=Y[train_idx]
        )
        # Ensure weights cover all classes (in case some are missing in a fold, though unlikely with StratifiedKFold)
        # But compute_class_weight returns weights for 'classes' param.
        # We need to map them to class indices 0..K-1.
        # Since we used LabelEncoder on Y, unique_classes should be integers.
        
        # Create a full weight tensor for all classes (handle potential missing classes in fold if N is small)
        num_classes_total = len(class_names) # or len(np.unique(Y)) globally
        weight_tensor = torch.ones(num_classes_total).to(DEVICE)
        
        for cls_idx, w in zip(unique_classes, weights):
            if cls_idx < num_classes_total:
                weight_tensor[cls_idx] = w
                
        crit_cls = nn.CrossEntropyLoss(weight=weight_tensor)
        
        best_acc = 0
        best_state = None
        
        for e in range(epochs_clf):
            clf.train()
            opt_clf.zero_grad()
            out = clf(z_tr)
            loss = crit_cls(out, y_tr)
            loss.backward()
            opt_clf.step()
            
            # Val (for early stop capability)
            clf.eval()
            with torch.no_grad():
                out_v = clf(z_val)
                acc_v = accuracy_score(y_val.cpu(), out_v.argmax(1).cpu())
                if acc_v > best_acc:
                    best_acc = acc_v
                    best_state = clf.state_dict()
        
        if best_state: clf.load_state_dict(best_state)
        
        # Tests
        clf.eval()
        with torch.no_grad():
            preds = clf(z_val).argmax(1).cpu().numpy()
            targets = y_val.cpu().numpy()
            
        fold_metrics['f1_macro'].append(f1_score(targets, preds, average='macro'))
        if not is_optuna:
            fold_metrics['f1_micro'].append(f1_score(targets, preds, average='micro'))
            fold_metrics['accuracy'].append(accuracy_score(targets, preds))
            fold_metrics['precision'].append(precision_score(targets, preds, average='macro', zero_division=0))
            fold_metrics['recall'].append(recall_score(targets, preds, average='macro', zero_division=0))
            
    if is_optuna:
        return np.mean(fold_metrics['f1_macro'])
    else:
        return {k: np.mean(v) for k, v in fold_metrics.items()}

# 

In [10]:
if __name__ == "__main__":
    if os.path.exists("../Data/expression_log.csv"):
        rna_df, meth_df, cnv_df, Y, class_names = load_raw_aligned_data()
        
        param_file = "HyperDNN_best_params.txt"
        with open(param_file, 'w') as f:
            f.write("Features | F1_Macro | Params\n")
            
        all_final_results = []
        
        print("\n" + "="*50)
        print("STARTING OPTUNA OPTIMIZATION FOR HYPERDNN")
        print("="*50)
        
        for n_feat in FEATURE_COUNTS:
            print(f"\n>>> Feature Count: {n_feat}")
            
            def obj(trial):
                params = {
                    # Latent space: keep it tight for small N
                    'latent_dim': trial.suggest_categorical('latent_dim', [16, 32, 48]), 
                    
                    # HyperNetwork parameters: low rank = better generalization
                    'hyper_hidden_dim': trial.suggest_categorical('hyper_hidden_dim', [32, 64]),
                    'embedding_dim': trial.suggest_categorical('embedding_dim', [16, 32]),
                    'rank': trial.suggest_categorical('rank', [4, 8, 16]),
                    
                    # Layers: Don't go too deep
                    'n_encoder_layers': trial.suggest_int('n_encoder_layers', 1, 2),
                    
                    # Learning Rates: Separate them to allow classifier focus
                    'lr_ae': trial.suggest_float('lr_ae', 1e-4, 1e-3, log=True),
                    'lr_clf': trial.suggest_float('lr_clf', 1e-3, 1e-2, log=True),
                    
                    # Regularization: Crucial for N=205
                    'dropout': trial.suggest_float('dropout', 0.3, 0.5),
                    'weight_decay': trial.suggest_float('weight_decay', 1e-4, 1e-1, log=True)
                }
                return run_cv_evaluation(params, n_feat, rna_df, meth_df, cnv_df, Y, class_names, is_optuna=True)
                
            study = optuna.create_study(direction="maximize")
            study.optimize(obj, n_trials=N_TRIALS)
            
            print(f"  Best F1: {study.best_value:.4f}")
            with open(param_file, 'a') as f:
                f.write(f"{n_feat} | {study.best_value:.4f} | {study.best_params}\n")
                
            # Final Eval
            res = run_cv_evaluation(study.best_params, n_feat, rna_df, meth_df, cnv_df, Y, class_names, is_optuna=False)
            res['n_features'] = n_feat
            all_final_results.append(res)
            
        # Summary
        df_res = pd.DataFrame(all_final_results)
        if not df_res.empty:
            cols = ['n_features', 'f1_macro', 'f1_micro', 'precision', 'recall', 'accuracy']
            print("\n" + "="*60)
            print("FINAL RESULTS SUMMARY")
            print("="*60)
            print(df_res[cols].round(4).to_string(index=False))
            df_res.to_csv("HyperDNN_results.csv", index=False)
    else:
        print("Data files not found in Data/ directory.")


>>> LOADING RAW ALIGNED DATA
  Phenotype Samples (filtered): 229
  Common Samples: 205
  Classes: ['Dedifferentiated liposarcoma' 'Fibromyxosarcoma' 'Leiomyosarcoma, NOS'
 'Undifferentiated sarcoma']

STARTING OPTUNA OPTIMIZATION FOR HYPERDNN

>>> Feature Count: 1500


[I 2026-01-20 13:08:15,690] A new study created in memory with name: no-name-6be522f9-c602-403d-9bf0-6d9697a1511c
[I 2026-01-20 13:09:14,675] Trial 0 finished with value: 0.5305458854361433 and parameters: {'latent_dim': 48, 'hyper_hidden_dim': 64, 'embedding_dim': 32, 'rank': 8, 'n_encoder_layers': 2, 'lr_ae': 0.00011698359439472504, 'lr_clf': 0.001968076194678931, 'dropout': 0.3744041636510419, 'weight_decay': 0.07743005826731766}. Best is trial 0 with value: 0.5305458854361433.
[I 2026-01-20 13:10:03,938] Trial 1 finished with value: 0.5483074547455352 and parameters: {'latent_dim': 32, 'hyper_hidden_dim': 64, 'embedding_dim': 16, 'rank': 4, 'n_encoder_layers': 1, 'lr_ae': 0.00014784034534556835, 'lr_clf': 0.001515727401050391, 'dropout': 0.4698751586627097, 'weight_decay': 0.00016925374587113348}. Best is trial 1 with value: 0.5483074547455352.
[I 2026-01-20 13:10:55,374] Trial 2 finished with value: 0.49052029952029946 and parameters: {'latent_dim': 16, 'hyper_hidden_dim': 32, 'em

  Best F1: 0.6356


[I 2026-01-20 13:21:55,748] A new study created in memory with name: no-name-b4972b0b-c23e-48ff-a61c-6afa472c87f0



>>> Feature Count: 2000


...

  Best F1: 0.6263


[I 2026-01-20 14:16:43,848] A new study created in memory with name: no-name-745259c5-8ef5-4ba7-ade0-f7707026fafa



>>> Feature Count: 5000


[I 2026-01-20 14:17:34,345] Trial 0 finished with value: 0.5567730175741011 and parameters: {'latent_dim': 16, 'hyper_hidden_dim': 64, 'embedding_dim': 32, 'rank': 8, 'n_encoder_layers': 2, 'lr_ae': 0.00018967159863015142, 'lr_clf': 0.0010712938233333107, 'dropout': 0.4713703473804157, 'weight_decay': 0.02138721962456301}. Best is trial 0 with value: 0.5567730175741011.
[I 2026-01-20 14:18:29,018] Trial 1 finished with value: 0.5605859930124637 and parameters: {'latent_dim': 16, 'hyper_hidden_dim': 32, 'embedding_dim': 32, 'rank': 4, 'n_encoder_layers': 2, 'lr_ae': 0.00020198246056321354, 'lr_clf': 0.00403599072469953, 'dropout': 0.30719136381779594, 'weight_decay': 0.008455869321796374}. Best is trial 1 with value: 0.5605859930124637.
[I 2026-01-20 14:19:22,233] Trial 2 finished with value: 0.556392352844096 and parameters: {'latent_dim': 32, 'hyper_hidden_dim': 64, 'embedding_dim': 16, 'rank': 8, 'n_encoder_layers': 1, 'lr_ae': 0.00012949569652501223, 'lr_clf': 0.003592088304833275, 

  Best F1: 0.6398

FINAL RESULTS SUMMARY
 n_features  f1_macro  f1_micro  precision  recall  accuracy
       1500    0.6243    0.7268     0.6423  0.6297    0.7268
       2000    0.6085    0.7024     0.6239  0.6097    0.7024
       2500    0.5950    0.7122     0.6114  0.6034    0.7122
       3000    0.5706    0.6829     0.5848  0.5727    0.6829
       4000    0.6196    0.7317     0.6481  0.6198    0.7317
       5000    0.5405    0.6537     0.5521  0.5432    0.6537
